# Setup

- uses functions from [utils](https://github.com/IanQS/blogpostcode/blob/master/src/Tf_Exploration/exploration/utils.py)

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import tensorflow as tf

tf.logging.set_verbosity(tf.logging.ERROR)
print(tf.__version__)

#tf.enable_eager_execution()
import numpy as np
import os
import datetime
import tqdm
import sys
import pprint

1.12.0


In [3]:
from Tf_Exploration.exploration.utils import FeatureProto, dataset_config

In [4]:
filename_list = []
for root, dirs, files in os.walk('processed_data/'):
    for f in files:
        if "tfrecord" in f:
            filename_list.append(os.path.join(root, f))
print(filename_list)
dataset = tf.data.TFRecordDataset(filename_list)

feature_proto = FeatureProto()
num_cpus = os.cpu_count()
features, labels = dataset_config(filenames=filename_list, batch_size=64, mapper=feature_proto.unpack, num_cpus=num_cpus,
                                 repeat=True)
columns = feature_proto.get_feature_columns()

['processed_data/tf_record_covtype_train_2019-01-06_01:00:00.tfrecord', 'processed_data/tf_record_covtype_test_2019-01-06_01:00:00.tfrecord']


# Define the network

# Custom Network

In [5]:
dense_tensor = tf.feature_column.input_layer(features=features, feature_columns=columns)
inputs = dense_tensor
for units in [256, 16]:
    dense_tensor = tf.layers.dense(dense_tensor, units, tf.nn.relu)
logits = tf.layers.dense(dense_tensor, 8)

# Verification
correct_pred = tf.equal(tf.argmax(logits, 1), tf.argmax(labels, 1))
accuracy = tf.reduce_mean(tf.cast(correct_pred, tf.float32))

# Training 
loss_op = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits_v2(logits=logits, labels=labels))
optimizer = tf.train.AdamOptimizer(learning_rate=0.001)
train_op = optimizer.minimize(loss_op)

# Canned Estimator

In [11]:
def input_fn_train(): # returns x, y (where y represents label's class index).
    return dataset_config(filenames=filename_list, batch_size=64, mapper=feature_proto.unpack, num_cpus=num_cpus)

def input_fn_eval(): # returns x, y (where y represents label's class index).
    return dataset_config(filenames=filename_list, batch_size=2048, mapper=feature_proto.unpack, num_cpus=num_cpus)


def evaluate(estimator):
    # Fit model.
    loss = estimator.evaluate(input_fn=input_fn_eval)
    print(loss)
    estimator.train(input_fn=input_fn_train)
    loss = estimator.evaluate(input_fn=input_fn_eval)
    print(loss)

In [14]:
from tensorflow.estimator import DNNClassifier

estimator = DNNClassifier(
    feature_columns=columns,
    n_classes=8,
    hidden_units=[256, 16],
    optimizer=lambda: tf.train.AdamOptimizer(
        learning_rate=tf.train.exponential_decay(
            learning_rate=0.1,
            global_step=tf.train.get_global_step(),
            decay_steps=1000,
            decay_rate=0.96)
    )
)

evaluate(estimator)

{'accuracy': 0.0, 'average_loss': 698.097, 'loss': 1428178.6, 'global_step': 0}
{'accuracy': 0.061537456, 'average_loss': 4.957756, 'loss': 10142.661, 'global_step': 9079}


# Baseline Estimator

In [13]:
from tensorflow.estimator import BaselineClassifier

classifier = BaselineClassifier(n_classes=8)

evaluate(classifier)

{'accuracy': 0.0, 'average_loss': 2.0794415, 'loss': 4254.1567, 'global_step': 0}
{'accuracy': 0.48759922, 'average_loss': 1.2731553, 'loss': 2604.6426, 'global_step': 9079}
